In [ ]:
import os

os.chdir("..")

In [ ]:
import geopandas as gpd
from src.inserts import data_inserts 
import polars as pl
import requests
di = data_inserts()

In [ ]:
# di.insert_regions()
# di.insert_states()
# di.insert_divisions()
# di.insert_county()
# di.insert_track()

In [ ]:
def pull_query(self, params: list, year: int) -> pl.DataFrame:
    # prepare custom census query
    param = ",".join(params)
    base = "https://api.census.gov/data"
    flow = "/acs/acs5/profile"
    url = f"{base}{year}{flow}?get={param}&for=county:*&in=state:*"
    df = pl.DataFrame(requests.get(url).json())

    # get names from DataFrame
    names = df.select(pl.col("column_0")).transpose()
    names = names.to_dicts().pop()
    names = dict((k, v.lower()) for k, v in names.items())

    # Pivot table
    df = df.drop("column_0").transpose()
    return df.rename(names).with_columns(year=pl.lit(year))


In [ ]:
# param = ",".join(params)
base = "https://api.census.gov/data"
# flow = "/acs/acs5/profile"
# url = f"{base}{year}{flow}?get={param}&for=county:*&in=state:*"
results = requests.get(base).json()

# # get names from DataFrame
# names = df.select(pl.col("column_0")).transpose()
# names = names.to_dicts().pop()
# names = dict((k, v.lower()) for k, v in names.items())

# # Pivot table
# df = df.drop("column_0").transpose()
# df = df.rename(names)
results

In [ ]:
pl.DataFrame(results.get("dataset"))

In [ ]:
df = pl.DataFrame(results.get("dataset"))

df = df.with_columns(
    dataset=pl.col("c_dataset").list.join(separator="-")
)
df = df.select("dataset", "c_vintage", "c_variablesLink","c_geographyLink", "title", "identifier")
df = df.sort(["dataset", "c_vintage"])
# df.write_csv("test.csv")

df

In [ ]:
urls = df.unique(subset="dataset",maintain_order=True, keep="last")
urls.with_columns(
    
)